# Visualizing Drought Data as Shape

In [ ]:
import pandas as pd
import geopandas as gpd

import seaborn as sns
import folium

In [ ]:
csv_file = 'DOAE spatial data/ภัยแล้ง/csv/droght_62.csv'
shp_file = 'DOAE spatial data/ภัยแล้ง/droght_62_4326/droght_62_4326.shp'

In [ ]:
drought_62 = pd.read_csv(csv_file)
drought_62.head()

In [ ]:
data = gpd.read_file(shp_file, encoding='utf-8')
data.head()

In [ ]:
data.shape

In [ ]:
data['des_th'].unique()

In [ ]:
def clean_des_th(s):
    if s.startswith('4-5'):
        return '4-5ครั้ง/10ปี'
    elif s.startswith('<=3'):
        return '<=3ครั้ง/10ปี'
    else:
        return '>=6ครั้ง/10ปี'

data['type'] = data['des_th'].apply(clean_des_th)

You can plot Shapely Polygon directly in Jupyter

In [ ]:
data.iloc[1].geometry

In [ ]:
m = folium.Map([14.069446, 100.603271], zoom_start=8, tiles='cartodbpositron')
folium.GeoJson(data[:100]).add_to(m)
folium.LatLngPopup().add_to(m)
m

## Dissolve Polygons

Plotting individual polygons can be slow. Merge/Dissolve them first (https://geopandas.org/aggregation_with_dissolve.html). Still dissolve can be slow.

In [ ]:
MAX_RECORDS = 1000

In [ ]:
droughts = data[:MAX_RECORDS].dissolve(by='type')

In [ ]:
droughts

In [ ]:
sum([len(poly.exterior.coords) for poly in droughts.iloc[0].geometry])

In [ ]:
s = droughts.iloc[0].geometry.simplify(0.001, preserve_topology=False)

In [ ]:
sum([len(poly.exterior.coords) for poly in s])

In [ ]:
droughts = droughts.reindex(['<=3ครั้ง/10ปี', '4-5ครั้ง/10ปี', '>=6ครั้ง/10ปี'])
droughts

In [ ]:
droughts.index

Set color for different labels using Seaborn color palette (https://seaborn.pydata.org/tutorial/color_palettes.html)

In [ ]:
colorscale = sns.color_palette("YlOrRd", 3).as_hex()
sns.palplot(colorscale)
colorscale

In [ ]:
colors = dict(zip(droughts.index, colorscale))

m = folium.Map([14.069446, 100.603271], zoom_start=5, tiles='cartodbpositron')

for i in range(droughts.shape[0]):
    folium.GeoJson(droughts.iloc[i:i+1], name=droughts.index[i],
                   style_function=lambda x: {'color' : colors[x['id']], 'fillColor' : colors[x['id']]}).add_to(m)
folium.LatLngPopup().add_to(m)
folium.LayerControl().add_to(m)
m

In [ ]:
def simplify_geometry(val):
    return val.simplify(0.001, preserve_topology=False)

droughts['geometry'] = droughts['geometry'].apply(simplify_geometry)

In [ ]:
colors = dict(zip(droughts.index, colorscale))

m = folium.Map([14.069446, 100.603271], zoom_start=5, max_zoom=11, tiles='cartodbpositron')

for i in range(droughts.shape[0]):
    folium.GeoJson(droughts.iloc[i:i+1], name=droughts.index[i],
                   style_function=lambda x: {'color' : colors[x['id']], 'fillColor' : colors[x['id']]}).add_to(m)
folium.LatLngPopup().add_to(m)
folium.LayerControl().add_to(m)
m

In [ ]:
m.save('./output/drought_62.html')